In [1]:
import flappy_bird_gym as fbg
from collections import namedtuple, deque, OrderedDict
from typing import Iterable
from IPython import display
import random
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import math

In [2]:
env = fbg.make("FlappyBird-v0")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [3]:
device

device(type='cuda')

In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
class DQN1(torch.nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN1, self).__init__()
        self.layer1 = torch.nn.Linear(n_observations,64)
        self.layer2 = torch.nn.Linear(64, 128)
        self.layer3 = torch.nn.Linear(128, 256)
        self.layer4 = torch.nn.Linear(256, 512) 
        self.layer5 = torch.nn.Linear(512, 512)
        self.layer6 = torch.nn.Linear(512, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = torch.nn.functional.relu(self.layer1(x))
        x = torch.nn.functional.relu(self.layer2(x))
        x = torch.nn.functional.relu(self.layer3(x))
        x = torch.nn.functional.relu(self.layer4(x))
        x = torch.nn.functional.relu(self.layer5(x))
        x = self.layer6(x)
        return x
    
class DQN(torch.nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = torch.nn.Linear(n_observations, 128)
        self.layer2 = torch.nn.Linear(128, 128)
        self.layer3 = torch.nn.Linear(128, n_actions)

    def forward(self, x):
        x = torch.nn.functional.relu(self.layer1(x))
        x = torch.nn.functional.relu(self.layer2(x))
        return self.layer3(x)

In [7]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
# agent = agent.Agent(BATCH_SIZE=32, MEMORY_SIZE=100000, GAMMA=0.99, input_dim=len_state, output_dim=n_actions, action_dim=n_actions, action_dict=action_dict, EPS_START=1.0, EPS_END=0.05, EPS_DECAY_VALUE=0.999995, TAU = 0.005, network_type='DuelingDQN', lr = 1e-4
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.05
LR = 1e-4
n_episodes = 200

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state = env.reset()
n_observations = len(state)

policy_net = DQN1(n_observations, n_actions).to(device)
target_net = DQN1(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = torch.optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        np.exp(-1. * steps_done / EPS_DECAY)
    #steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            out = policy_net(state)#.max(1).indices
            #print(f'select_action: {out}')
            return out.max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], dtype=torch.long, device=device)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [8]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return None
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None]).view(-1, 2)
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = torch.nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

    return loss

In [ ]:
num_episodes = 10000 #n_episodes
old_score = 0
obs = 0
ns = 0

for episode in range(num_episodes):
    old_score = 0
    state = env.reset()
    state = torch.tensor(state, device=device, dtype=torch.float32).unsqueeze(dim=0)
    loss_ls = []
    reward_ls = []

    for t in itertools.count():
        action = select_action(state)
        observation, reward, done, info = env.step(action.item())

        #reward = torch.tensor([[reward]], device=device)

        if done:
            next_state = None
            reward = torch.tensor([[-10]], device=device)
        else:
            next_state = torch.tensor(observation, device=device, dtype=torch.float32).unsqueeze(dim=0)
            if type(next_state) == float:
                print('sdcfnsd')
                next_state = None

            if info['score'] == old_score:
                if next_state is not None:
                    reward = torch.tensor([[reward * 0.1 * (1 - np.abs(next_state[:, 1].item()))]], device=device)
                else:
                    reward = torch.tensor([[2]], device=device)
                # set reward depending on how close the bird is to the hole
            elif info['score'] > old_score:
                reward = torch.tensor([[10]], device=device)

        reward_ls.append(reward)
        memory.push(state, action, next_state, reward)
        state = next_state
        
        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        if loss:
            loss_ls.append(loss)

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break
    if episode % 5 == 0:
        print(f'episode {episode} | loss {torch.tensor(loss_ls).mean()} | reward {torch.tensor(reward_ls).sum()}')

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()


episode 0 | loss nan | reward -2.8759765624999987


/home/german/Documents/unik/programming-for-ai/main_project/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([128, 1, 128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


episode 5 | loss 0.09418888137324799 | reward -4.196289062500001
episode 10 | loss 0.09758376514699647 | reward -4.7701171874999995
episode 15 | loss 0.09849761060302346 | reward -2.9857421875000005
episode 20 | loss 0.12431216550942262 | reward -3.8107421875
episode 25 | loss 0.10532357206603905 | reward -4.846875000000001
episode 30 | loss 0.10942564919717154 | reward -3.98046875
episode 35 | loss 0.10752296654154159 | reward -5.0523437499999995
episode 40 | loss 0.12140483025312938 | reward -3.7548828124999996
episode 45 | loss 0.11133056972605423 | reward -4.5970703125
episode 50 | loss 0.12286278077514227 | reward -3.993359375
episode 55 | loss 0.11041345743465282 | reward -4.129296875
episode 60 | loss 0.1359245470341389 | reward -4.128710937499999
episode 65 | loss 0.1284952469902565 | reward -4.398046875
episode 70 | loss 0.1334951618328358 | reward -3.3490234375000005
episode 75 | loss 0.12458125536065698 | reward -3.2681640624999995
episode 80 | loss 0.14483517442374674 | rew

In [118]:
state = env.reset()
state = torch.tensor(state, device=device).unsqueeze(0)
act = select_action(state)
env.step(act.item())

(array([ 1.64236111, -0.0859375 ]), 1, False, {'score': 0})

In [54]:
transitions = memory.sample(1000)
batch = Transition(*zip(*transitions))

states_batch = torch.cat(batch.state, dim=0)
action_batch = torch.cat(batch.action, dim=0)
rewards_batch = torch.cat(batch.reward)

non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                        batch.next_state)), device=device, dtype=torch.bool)
non_final_next_states = torch.cat([s for s in batch.next_state
                                            if s is not None], dim=0).view(-1, 2)

TypeError: expected Tensor as element 355 in argument 0, but got float

In [114]:
next_state[:, [1]]

tensor([[0.0391]], device='cuda:0')

In [66]:
import math
math.isnan([s for s in batch.next_state if s is not None][355])

True

In [125]:
for i in range(20):
    res = env.step(1)
    print(res[0])

[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]
[0.26736111 0.54492188]


In [126]:
env.step(1)

(array([0.26736111, 0.54492188]), 1, True, {'score': 0})